# Discriminator

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import required libraries

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from datetime import date
from itertools import product
import os
import torchvision.models as tmodels
from functools import partial
import collections
from sklearn.svm import OneClassSVM

### Define functions

In [ ]:
def get_mean_channels(batched_outputs):
    channel_means = []
    for single_output in batched_outputs:
        channel_means.append([channel.mean() for channel in single_output])
    return torch.tensor(channel_means)

### Prepare Data

In [ ]:
trg_dataset_ref = 1

# Retrieve normalisation parameters 

norm_param_df = pd.read_csv('/content/drive/MyDrive/KASHIKO/DATASET/TRG_DATASET_NORM_PARAM.csv')

meanR = norm_param_df.loc[norm_param_df["Dataset"] == str(trg_dataset_ref), "meanR"].item()
meanG = norm_param_df.loc[norm_param_df["Dataset"] == str(trg_dataset_ref), "meanG"].item()
meanB = norm_param_df.loc[norm_param_df["Dataset"] == str(trg_dataset_ref), "meanB"].item()

stdR = norm_param_df.loc[norm_param_df["Dataset"] == str(trg_dataset_ref), "stdR"].item()
stdG = norm_param_df.loc[norm_param_df["Dataset"] == str(trg_dataset_ref), "stdG"].item()
stdB = norm_param_df.loc[norm_param_df["Dataset"] == str(trg_dataset_ref), "stdB"].item()

In [ ]:
dataset = datasets.ImageFolder(
    '/content/drive/MyDrive/KASHIKO/DATASET/TRG_1_FINAL',
    transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((meanR, meanG, meanB), (stdR, stdG, stdB))
    ])
)

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 1000, 1000])

loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=50,
        shuffle=True,
        num_workers=2,
        drop_last=True)

### Load models

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.bn1 = nn.BatchNorm2d(12)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.bn2 = nn.BatchNorm2d(24)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(24*53*53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1,24*53*53)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net1 = Net()

In [ ]:
state_dict1 = torch.load('/content/drive/MyDrive/KASHIKO/MODELS/model_2021-05-29_12:16:11_ trg_dataset1 batch_size=100 learning_rate=0.001 scheduler_step_size=5 scheduler_gamma=1 weight_decay=0 epoch_number=14 accuracy=98.0.pth')

In [ ]:
net1.load_state_dict(state_dict1)

### Generate activation files for SVM Model Training

In [ ]:
activations1 = collections.defaultdict(list)
def save_activation1(name, mod, inp, out1):
    activations1[name].append(out1.cpu())

In [ ]:
for name, m in net1.named_modules():
    if type(m)==nn.Conv2d:
        m.register_forward_hook(partial(save_activation1, name))
    elif type(m)==nn.Linear:
        m.register_forward_hook(partial(save_activation1, name))
    elif type(m)==nn.BatchNorm2d:
        m.register_forward_hook(partial(save_activation1, name))

In [ ]:
# Forward pass of the full dataset
with torch.no_grad():
  for images, labels in loader:
    net1.eval()
    out1 = net1(images)

activations1 = {name: torch.cat(outputs, 0) for name, outputs in activations1.items()}
torch.save(activations1,'/content/drive/MyDrive/KASHIKO/MODELS/SVM_training_activations1.pt')

### Generate activation files for the In-Distribution Dataset

In [ ]:

activationsID = collections.defaultdict(list)
def save_activationID(name, mod, inp, outID):
    activationsID[name].append(outID.cpu())

In [ ]:
ID_valset = datasets.ImageFolder('/content/drive/MyDrive/KASHIKO/DATASET/ID2',
    transforms.Compose([transforms.ToTensor(),transforms.Normalize((meanR, meanG, meanB), (stdR, stdG, stdB))]))

In [ ]:
IDloader = torch.utils.data.DataLoader(
        ID_valset,
        batch_size=10,
        shuffle=True,
        num_workers=2,
        drop_last=True)

In [ ]:
for name, m in net1.named_modules():
    if type(m)==nn.Conv2d:
        # partial to assign the layer name to each hook
        m.register_forward_hook(partial(save_activationID, name))
    elif type(m)==nn.Linear:
        m.register_forward_hook(partial(save_activationID, name))
    elif type(m)==nn.BatchNorm2d:
        m.register_forward_hook(partial(save_activationID, name))
        
with torch.no_grad():
  for images, labels in IDloader:
    net1.eval()
    outID = net1(images)

activationsID = {name: torch.cat(outputs, 0) for name, outputs in activationsID.items()}
torch.save(activationsID,'/content/drive/MyDrive/KASHIKO/MODELS/SVM_testing_activationsID2.pt')

### Generate activation files for the Out-Of-Distribution Dataset

In [ ]:
OOD_valset = datasets.ImageFolder('/content/drive/MyDrive/KASHIKO/DATASET/OOD',
    transforms.Compose([transforms.ToTensor(),transforms.Normalize((meanR, meanG, meanB), (stdR, stdG, stdB))]))

In [ ]:
OODloader = torch.utils.data.DataLoader(
        OOD_valset,
        batch_size=10,
        shuffle=True,
        num_workers=2,
        drop_last=True)

In [ ]:
#activations.clear()
activationsOOD = collections.defaultdict(list)
def save_activationOOD(name, mod, inp, outOOD):
    activationsOOD[name].append(outOOD.cpu())
    
for name, m in net1.named_modules():
    if type(m)==nn.Conv2d:
        # partial to assign the layer name to each hook
        m.register_forward_hook(partial(save_activationOOD, name))
    elif type(m)==nn.Linear:
        m.register_forward_hook(partial(save_activationOOD, name))
    elif type(m)==nn.BatchNorm2d:
        m.register_forward_hook(partial(save_activationOOD, name))
        
with torch.no_grad():
  for images, labels in OODloader:
    net1.eval()
    outOOD = net1(images)

activationsOOD = {name: torch.cat(outputs, 0) for name, outputs in activationsOOD.items()}
torch.save(activationsOOD,'/content/drive/MyDrive/KASHIKO/MODELS/SVM_testing_activationsOOD.pt')

### Load Activation files

In [ ]:
ACT1 = collections.defaultdict(list)
ACT1 = torch.load('/content/drive/MyDrive/KASHIKO/MODELS/SVM_training_activations1.pt')

ACTID = collections.defaultdict(list)
ACTID = torch.load('/content/drive/MyDrive/KASHIKO/MODELS/SVM_testing_activationsID2.pt')

ACTOOD = collections.defaultdict(list)
ACTOOD = torch.load('/content/drive/MyDrive/KASHIKO/MODELS/SVM_testing_activationsOOD.pt')

# Sanity check of the size/shape of the Activation files
for k,v in ACT1.items():
    print (k, v.size())

for k,v in ACTID.items():
    print (k, v.size())

for k,v in ACTOOD.items():
    print (k, v.size())

### Train SVM Model for each layer and compare performance with ID and OOD Datasets

In [ ]:
# CONV1
batched_output = get_mean_channels(ACT1['conv1'])
model = OneClassSVM(gamma='auto', nu=0.001).fit(batched_output)

id_features = get_mean_channels(ACTID['conv1'])
ood_features = get_mean_channels(ACTOOD['conv1'])

data = np.vstack((id_features,ood_features))
preds = model.predict(data)

id_error = 1 - np.count_nonzero(preds[:100] == 1) / 100
ood_error = 1 - np.count_nonzero(preds[100:] == -1) / 100
detection_errors = (id_error + ood_error) / 2

print(f'ID error : {id_error}')
print(f'ood_error : {ood_error}')
print(f'Detection error for layer : {detection_errors}')

In [ ]:
# BN1
batched_output = get_mean_channels(ACT1['bn1'])
model = OneClassSVM(gamma='auto', nu=0.001).fit(batched_output)

id_features = get_mean_channels(ACTID['bn1'])
ood_features = get_mean_channels(ACTOOD['bn1'])

data = np.vstack((id_features,ood_features))
preds = model.predict(data)

id_error = 1 - np.count_nonzero(preds[:100] == 1) / 100
ood_error = 1 - np.count_nonzero(preds[100:] == -1) / 100
detection_errors = (id_error + ood_error) / 2

print(f'ID error : {id_error}')
print(f'ood_error : {ood_error}')
print(f'Detection error for layer : {detection_errors}')

In [ ]:
# CONV2
batched_output = get_mean_channels(ACT1['conv2'])
model = OneClassSVM(gamma='auto', nu=0.001).fit(batched_output)

id_features = get_mean_channels(ACTID['conv2'])
ood_features = get_mean_channels(ACTOOD['conv2'])

data = np.vstack((id_features,ood_features))
preds = model.predict(data)

id_error = 1 - np.count_nonzero(preds[:100] == 1) / 100
ood_error = 1 - np.count_nonzero(preds[100:] == -1) / 100
detection_errors = (id_error + ood_error) / 2

print(f'ID error : {id_error}')
print(f'ood_error : {ood_error}')
print(f'Detection error for layer : {detection_errors}')

In [ ]:
# BN2
batched_output = get_mean_channels(ACT1['bn2'])
model = OneClassSVM(gamma='auto', nu=0.001).fit(batched_output)

id_features = get_mean_channels(ACTID['bn2'])
ood_features = get_mean_channels(ACTOOD['bn2'])

data = np.vstack((id_features,ood_features))
preds = model.predict(data)

id_error = 1 - np.count_nonzero(preds[:100] == 1) / 100
ood_error = 1 - np.count_nonzero(preds[100:] == -1) / 100
detection_errors = (id_error + ood_error) / 2

print(f'ID error : {id_error}')
print(f'ood_error : {ood_error}')
print(f'Detection error for layer : {detection_errors}')

In [ ]:
# FC1
model = OneClassSVM(gamma='auto', nu = 0.001).fit(ACT1['fc1'])

id_features = ACTID['fc1']
ood_features = ACTOOD['fc1']

data = np.vstack((id_features,ood_features))
preds = model.predict(data)

id_error = 1 - np.count_nonzero(preds[:100] == 1) / 100
ood_error = 1 - np.count_nonzero(preds[100:] == -1) / 100
detection_errors = (id_error + ood_error) / 2

print(f'ID error : {id_error}')
print(f'ood_error : {ood_error}')
print(f'Detection error for layer : {detection_errors}')

In [ ]:
# FC2
model = OneClassSVM(gamma='auto', nu = 0.001).fit(ACT1['fc2'])

id_features = ACTID['fc2']
ood_features = ACTOOD['fc2']

data = np.vstack((id_features,ood_features))
preds = model.predict(data)

id_error = 1 - np.count_nonzero(preds[:100] == 1) / 100
ood_error = 1 - np.count_nonzero(preds[100:] == -1) / 100
detection_errors = (id_error + ood_error) / 2

print(f'ID error : {id_error}')
print(f'ood_error : {ood_error}')
print(f'Detection error for layer : {detection_errors}')